# Ungraded Lab: Database Design Lab

## 📋 Overview 
Welcome to the Database Design Lab! In this hands-on session, you'll step into the role of a database administrator for BookCycle, our fictional used bookstore chain. Your mission is to design and implement a small but robust database schema that will help BookCycle manage its inventory, sales, and customer data efficiently.<br>

By the end of this lab, you'll have created a well-structured database schema with proper constraints and indexes, ensuring data integrity and optimizing query performance. This experience will give you practical insights into real-world database design challenges and solutions.


## 🎯 Learning Outcomes
By the end of this lab, you will be able to:

- Design and implement a relational database schema for a real-world scenario
- Create tables with appropriate primary keys, foreign keys, and constraints
- Implement indexes to improve query performance
- Use SQL to define and modify database structures


## 📚 Dataset Information
We'll be working with the following datasets for BookCycle:
- <b>books.csv:</b> Contains information about the books in inventory
- <b>transactions.csv:</b> Records of book sales transactions

We’ll be creating the following datasets for BookCycle:
- <b>suppliers.csv:</b> Information about book suppliers
- <b>wishlists.csv:</b> Customer wishlists for future purchases

## 🖥️ Activities

### Activity 1: Connecting to the Database and Exploring Existing Tables

Let's start by connecting to the BookCycle database and examining its current structure.

<b>Step 1:</b> Import necessary libraries and establish a database connection:

In [1]:
import sqlite3
import pandas as pd

# Setting up the database
from db_setup_2 import setup_database
setup_database() 

# Connect to the BookCycle database
conn = sqlite3.connect('bookcycle.db')
cursor = conn.cursor()

# Function to execute SQL queries and display results
def execute_query(query):
    df = pd.read_sql_query(query, conn)
    display(df)

✅ Database setup complete: Tables created and populated with data!


<b>Step 2:</b> Explore the existing tables in the database:

In [2]:
query = """
SELECT name FROM sqlite_master WHERE type='table';
"""
execute_query(query)


,name
0,books
1,customers
2,transactions


<b>Step 3:</b> Examine the structure of the 'books' table:

In [3]:
query = """
PRAGMA table_info(books);
"""
execute_query(query)


,cid,name,type,notnull,dflt_value,pk
0,0,book_id,TEXT,0,None,0
1,1,title,TEXT,0,None,0
2,2,author,TEXT,0,None,0
3,3,isbn,INTEGER,0,None,0
4,4,genre,TEXT,0,None,0
5,5,condition,TEXT,0,None,0
6,6,purchase_price,REAL,0,None,0
7,7,list_price,REAL,0,None,0
8,8,date_acquired,TEXT,0,None,0
9,9,current_location,TEXT,0,None,0


<b>💡 Tip:</b> The PRAGMA table_info() command is specific to SQLite and provides information about table columns, including their names, types, and constraints.

### Activity 2: Creating the Suppliers Table 

Now, let's create a new table to store information about BookCycle's suppliers.

<b>Step 1:</b> Design the suppliers table schema:
```
CREATE TABLE IF NOT EXISTS suppliers (
    supplier_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    contact_info TEXT,
    address TEXT
);
```

<b>Step 2:</b> Execute the CREATE TABLE statement:


In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS suppliers (
    supplier_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    contact_info TEXT,
    address TEXT
);
""")
conn.commit()


<b>Step 3:</b> Verify the table creation:

In [5]:
query = """
PRAGMA table_info(suppliers);
"""
execute_query(query)


,cid,name,type,notnull,dflt_value,pk
0,0,supplier_id,INTEGER,0,None,1
1,1,name,TEXT,1,None,0
2,2,contact_info,TEXT,0,None,0
3,3,address,TEXT,0,None,0


<b>💡 Tip:</b> Always use IF NOT EXISTS when creating tables to avoid errors if the table already exists.

### Activity 3: Adding Foreign Key Constraints

Let's add a foreign key to the 'books' table to establish a relationship with the 'suppliers' table.

<b>Step 1:</b> First, let's examine the current <b>books</b> table structure:

In [6]:
query = """
PRAGMA table_info(books);
"""
execute_query(query)


,cid,name,type,notnull,dflt_value,pk
0,0,book_id,TEXT,0,None,0
1,1,title,TEXT,0,None,0
2,2,author,TEXT,0,None,0
3,3,isbn,INTEGER,0,None,0
4,4,genre,TEXT,0,None,0
5,5,condition,TEXT,0,None,0
6,6,purchase_price,REAL,0,None,0
7,7,list_price,REAL,0,None,0
8,8,date_acquired,TEXT,0,None,0
9,9,current_location,TEXT,0,None,0


<b>Step 2:</b> Now, let's add a <b>supplier_id</b> column to the <b>books</b> table:

In [7]:
cursor.execute("""
ALTER TABLE books ADD COLUMN supplier_id INTEGER;
""")
conn.commit()

<b>Step 3:</b> Add a foreign key constraint:


In [8]:
cursor.execute("CREATE TABLE IF NOT EXISTS books_temp AS SELECT * FROM books");
cursor.execute("DROP TABLE books");

cursor.execute("""
CREATE TABLE books (
    book_id TEXT PRIMARY KEY,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    isbn INTEGER,
    genre TEXT,
    condition TEXT,
    purchase_price REAL,
    list_price REAL,
    date_acquired TEXT,
    current_location TEXT,
    quantity INTEGER,
    supplier_id INTEGER,
    FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
    )
""");


cursor.execute("INSERT INTO books SELECT * FROM books_temp");

cursor.execute("DROP TABLE books_temp");
conn.commit()

<b>💡 Tip:</b> In SQLite, adding a foreign key constraint requires recreating the table. In other database systems, you might be able to add the constraint directly with an ALTER TABLE statement.

<b>Step 4:</b> Verify the new table structure:

In [9]:
query = """
PRAGMA table_info(books);
"""
execute_query(query)


,cid,name,type,notnull,dflt_value,pk
0,0,book_id,TEXT,0,None,1
1,1,title,TEXT,1,None,0
2,2,author,TEXT,1,None,0
3,3,isbn,INTEGER,0,None,0
4,4,genre,TEXT,0,None,0
5,5,condition,TEXT,0,None,0
6,6,purchase_price,REAL,0,None,0
7,7,list_price,REAL,0,None,0
8,8,date_acquired,TEXT,0,None,0
9,9,current_location,TEXT,0,None,0


### Activity 4: Creating Indexes for Performance 

To improve query performance, let's add some indexes to our tables.

<b>Step 1:</b>  Create an index on the 'isbn' column of the 'books' table:

In [10]:
cursor.execute("""
CREATE INDEX idx_books_isbn ON books(isbn);
""")

conn.commit()

<b>Step 2:</b> Create a compound index, an index that includes multiple columns, helping optimize queries that filter or sort by those columns together, on 'author' and 'title' columns:


In [11]:
cursor.execute("""
CREATE INDEX idx_books_author_title ON books(author, title);
""")
conn.commit()

<b>Step 3:</b>  Verify the created indexes:

In [12]:
query = """
SELECT name FROM sqlite_master WHERE type='index' AND tbl_name='books';
"""
execute_query(query)


,name
0,sqlite_autoindex_books_1
1,idx_books_isbn
2,idx_books_author_title


<b>💡 Tip:</b> Indexes can significantly improve query performance, but they also take up space and slow down data modifications. Choose your indexes wisely based on your most common queries.

### Activity 5: Creating the Wishlists Table
Finally, let's create a table to store customer wishlists.

<b>Step 1:</b> Design and create the wishlists table:

In [13]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS wishlists (
    wishlist_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    book_id INTEGER,
    date_added DATE,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);
""")
conn.commit()

<b>Step 2:</b> Verify the table creation:

In [14]:
query = """
PRAGMA table_info(wishlists);
"""
execute_query(query)


,cid,name,type,notnull,dflt_value,pk
0,0,wishlist_id,INTEGER,0,None,1
1,1,customer_id,INTEGER,0,None,0
2,2,book_id,INTEGER,0,None,0
3,3,date_added,DATE,0,None,0


#### Close the Connection
It's good practice to close the database connection when you're done

In [15]:
# Close the database connection 
# Here, we are dropping these tables for a streamlined lab experience. 
# In real-world, its not always necessary to drop table once your analysis is complete
cursor.execute("DROP TABLE wishlists");
cursor.execute("DROP TABLE suppliers");

conn.commit()
conn.close()

### ⚙️ Test Your Work:
1. Try inserting a sample record into the suppliers table
2. Attempt to insert a book with a non-existent supplier_id
3. Query the books table using the newly created indexes

## ✅ Success Checklist
- Created suppliers table with appropriate columns and constraints
- Added foreign key constraint to books table linking to suppliers
- Created indexes on the books table for improved query performance
- Created wishlists table with appropriate foreign key constraints
- Verified all table structures and relationships


## 🔍 Common Issues & Solutions 

- Problem: Error when adding foreign key constraint 
    - Solution: Ensure that the referenced table (suppliers) exists and has a matching primary key column

- Problem: Index creation fails 
    - Solution: Check for typos in table or column names, and ensure the table exists


## ➡️ Summary
Congratulations on completing the Database Design Lab! You've successfully designed and implemented a functional database schema for BookCycle, demonstrating your ability to translate real-world business requirements into an efficient and well-structured database. This hands-on experience in creating tables, establishing relationships, and optimizing performance will serve as a solid foundation for your future database design projects.

### 🔑 Key Points
- Proper schema design is crucial for data integrity and query performance
- Foreign key constraints help maintain relationships between tables
- Indexes can significantly improve query performance for frequently accessed data
- Always consider the trade-offs when adding indexes (improved read performance vs. slower write operations)
